In [4]:
!pip install pandas-profiling==2.7.1

In [5]:
import os, sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling
import scipy.stats  as stats
from scipy.stats import chi2
from scipy.stats import chi2_contingency

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1. Entendimiento del negocio y de los datos 




In [6]:
df = pd.read_excel("Datos_parcial2.xlsx", index_col ="ID")
df

,COD_SAL,VIA,CANTI,PBK,PNK,FOBPES,FOBDOL,FLETES,SEGURO,OTROSG,PAIS
ID,,,,,,,,,,,
9087,BOG,4,140.00,0.02660,0.02460,5.038243,1.67604,517.00,0.00,0.00,PAISES BAJOS
3481,BOG,4,142.00,0.06101,0.05636,4.309442,1.43917,45.24,1.09,0.00,ECUADOR
9368,BOG,4,1.00,0.00140,0.00100,0.274905,0.09251,0.00,NaN,0.00,PANAMA
21953,IPI,3,2.00,0.45793,0.39397,38.356037,12.72600,269.54,NaN,89.84,ECUADOR
14218,BUN,1,5328.00,5.91942,5.32800,17.028346,5.62038,0.00,NaN,0.00,BRASIL
...,...,...,...,...,...,...,...,...,...,...,...
17669,BUN,1,610.05,0.61005,0.61005,5.202518,1.72487,0.00,NaN,0.00,PANAMA
19754,BUN,1,89.00,0.01431,0.01289,2.002604,0.66098,0.00,NaN,0.00,BOLIVIA
2260,BOG,4,2.00,0.00061,0.00039,0.011849,0.00400,0.00,NaN,0.00,COSTA RICA


In [ ]:
pandas_profiling.ProfileReport(df)

## Describa el resultado del perfilamiento de datos e indique el tipo de cada variable.
Revisando el perfilamiento de datos se puede evidenciar que el 1.4% de las filas son duplicadas, dadas las variables considero que se deberían remover pues es bastante improbable que se presenten tales coincidencias dadas las unidades de las variables como Peso bruto y Peso neto de la mercancía, columnas como _OTROSG_ y _SEGURO_ cuentan con registros nulos que, de acuerdo al diccionario y la frecuencia de los valores presentes en estas variables puede deberse a valores no marcados como 0. Asimismo, se encontró que variables como _FLETES_ y las previamente mencionadas _SEGURO_ y _OTROSG_ cuentan con gran cantidad de valores 0 superior al 70%


In [ ]:
for columna in df.columns:
  print(columna)
  print(df[columna].unique(), len(df[columna].unique()))

Vemos que para variables como _COD_SAL_ y _VIA_ hay menos valores que los posibles mencionados en el diccionario y que sin embargo son posibles. También vemos que para la variable PAIS hay más valores posibles que los mencionados en el diccionario por lo que es mejor removerlos pues no son esperados. También es posible ver que el tipo de la variable _CANTI_ no corresponde al esperado pues debería ser una variable discreta y además presenta valores ireales que serán especificados más adelante.

In [ ]:
df.dtypes

## Describa el entendimiento que tiene del negocio a partir de los datos suministrados. 

Los datos están relacionados con exportaciones realizadas en Colombia en el mes de Diciembre en un año específico. 
En el dataset hay 9 variables numéricas y 3 categóricas que describen una exportación.
* ID	ID de la exportación
* COD_SAL Lugar de donde sale la mercancía.
* VIA Vía de transporte.
* CANTI Número de unidades del producto exportado
* PBK Peso bruto de la mercancía (En Kilos)
* PNK Peso neto de la mercancía (En Kilos)
* FOBDOL Valor total de la mercancía (en miles de dólares)
* FOBPES Valor total de la mercancía (en millones de pesos)
* FLETES Valor fletes de la mercancía (En pesos)
* SEGURO Valor asociado a seguros de la mercancía
* OTROSG Otros gastos de la mercancía
* PAIS País destino de la mercancía exportada

# 2. Análisis y propuestas accionables.

## Hipótesis de interés.
1. La ubicación del lugar de dónde sale la mercancía está relacionada con el país de destino, entre más adentro geográficamente se encuentre el lugar de salida, más cerca se encuentra el país de destino.
2. El peso bruto está relacionado de manera directa con el valor total de la mercancía en millones de pesos, entre más pesada sea la mercancía, más va a costar.
3. El valor de otros gastos de la mercancía está relacionado con la vía de transporte usada para la exportación, entre menos común sea mayor va a ser el valor de los otros gastos.
4. El número de unidades del producto exportado está relacionado con la ubicación del lugar de dónde sale la mercancía, entre más costera sea la ubicación del lugar de dónde sale la mercancía más unidades se van a exportar

### Hipótesis 1
La ubicación del lugar de dónde sale la mercancía está relacionada con el país de destino, entre más adentro geográficamente se encuentre el lugar de salida, más cerca se encuentra el país de destino.

In [ ]:
df2 = df.groupby(['PAIS', 'COD_SAL'])['PAIS'].count().unstack('COD_SAL').fillna(0)
ax = df2.plot(kind='bar', stacked=True,figsize=(15,15))
NUM_COLORS = 20

cm = plt.get_cmap('gist_rainbow')
ax.set_title("Cantidad de exportaciones por país de destino y región de salida")

Podemos ver que hay regiones como CTG o BUN que participan activamente en algunas exportaciones mientras que en otras no lo que podría ayudarnos a dar una idea de la validez de la hipótesis

### Hipótesis 2
El peso bruto está relacionado de manera directa con el valor total de la mercancía en millones de pesos, entre más pesada sea la mercancía, más va a costar.

In [ ]:
df.plot.scatter(x='FOBPES', y='PBK')

Acá podemos ver que hay una especie de relación lineal entre el peso bruto y el valor total de la mercancía en millones de pesos

### Hipótesis 3.
El valor de otros gastos de la mercancía está relacionado con la vía de transporte usada para la exportación, entre menos común sea mayor va a ser el valor de los otros gastos.


In [ ]:
sns.violinplot("VIA","OTROSG",data=df)

### Hipótesis 4.
El número de unidades del producto exportado está relacionado con la ubicación del lugar de dónde sale la mercancía, entre más costera sea la ubicación del lugar de dónde sale la mercancía más unidades se van a exportar

In [ ]:
ax = df.groupby("COD_SAL")["CANTI"].count().plot(kind="bar")
ax.set_ylabel("CANTIDAD")

Acá podemos ver que hay algunas regiones de salida que tienen una gran cantidad de exportaciones y entre ellas hay presentes varias costeras como Cartagena.

## Problemas de calidad de los datos que puedan afectar los análisis propuestos en las hipótesis 

La cantidad de valores nulos mencionada previamente podría afectar. Sin embargo, dado el bajo porcentaje su impacto al eliminarlas sería leve por lo que es lo que haré, también sería conveniente validar con un experto en el negocio la razón de estos. Asimismo, los valores duplicados como se mencionó previamente no puede afectar notablemente los análisis propuestos en las hipótesis pues su porcentaje es poco y considero que a pesar de lo anterior y dadas las unidades de las variables las probabilidades de presencia de estas coincidencias son nulas por lo que es mejor eliminarlas. 
Todas las variables corresponden exactamente a la realidad planteada en el diciconario de datos, a excepción de las variables
* PAIS que cuenta con 4 registros fuera de los mencionados en el diccionario.
* PBK que cuenta con valores en 0
* CANTI que cuenta con valores decimales

## Ajustes sobre los datos

Inicialmente se eliminarán las filas que en la variable país contengan a alguno de los países no especificados en el diccionario

También se eliminarán las filas que contengan en la variable PBK el valor de 0 y que en canti cuenten con valores decimales y se cambiará el tipo a Int.


In [ ]:
aRemov = ["BRASIL","HONDURAS","VENEZUELA","REINO UNIDO"] 
for a in aRemov:
  df = df[df.PAIS!=a]
print(df.PAIS.unique(),len(df.PAIS.unique()))

In [ ]:
df = df[df["PBK"]>0]
df = df[df["CANTI"]%1==0]
df["CANTI"] = df["CANTI"].astype(int)

In [ ]:
df = df.drop_duplicates()

## Validación de hipótesis planteadas

### Hipótesis 1
La ubicación del lugar de dónde sale la mercancía está relacionada con el país de destino, entre más adentro geográficamente se encuentre el lugar de salida, más cerca se encuentra el país de destino.

In [ ]:
print(df["COD_SAL"].unique())

In [ ]:
def freq_relativFil(col,df):
  Tabla = pd.crosstab(index=df[col],columns=df["COD_SAL"],margins=True)
  cols = list(Tabla.columns)
  cols[-1] = "Total_"+col
  Tabla.columns = cols
  idxs = list(Tabla.index)
  idxs[-1] = "Total_y"
  Tabla.index = idxs
  return Tabla.div(Tabla["Total_"+col], axis=0)

Tabla = freq_relativFil('PAIS',df)
Tabla

ax = (Tabla[df["COD_SAL"].unique()]).plot(kind='bar',figsize=(15,15),width = 0.8,edgecolor=None, stacked=True)
plt.legend(labels=Tabla.columns,fontsize= 14)
plt.title("% de exportaciones a cada destino por región de origen",fontsize= 16)

plt.xticks(fontsize=14, rotation=90)
for spine in plt.gca().spines.values():
    spine.set_visible(False)
plt.yticks([])

# Add this loop to add the annotations
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate('{:.0%}'.format(height), (x, y + height + 0.01))

In [ ]:
def contingencia(col,df):
  Tabla = pd.crosstab(index=df[col],columns=df["COD_SAL"])
  return Tabla

def chiquad(Tabla, prob = 0.95):
  estadistico, p_value, grados_libertad, freq_esperadas = chi2_contingency(Tabla)
  print(f"El estadístico de prueba es {estadistico}, el p_value es {p_value}, los grados de libertad {grados_libertad} y las frecuencias esperadas son:\n {freq_esperadas}")
  print("Interpretando p_value con probabilidad del {}%".format(prob*100))
  alpha = 1.0 - prob
  if p_value <= alpha:
    print('Dependientes (Se rechaza H0)')
  else:
    print('No son dependientes (No se puede rechazar H0)')
  print("Interpretando estadístico de prueba con probabilidad del {0}% y {1} grados de libertad".format(prob*100,grados_libertad))
  # interpret test-statistic
  critical = chi2.ppf(prob, grados_libertad)
  if abs(estadistico) >= critical:
    print('Dependientes (Se rechaza H0)')
  else:
    print('No son dependientes (No se puede rechazar H0)')
Tabla = contingencia("PAIS",df)
Tabla

In [ ]:
chiquad(Tabla)

Las variables COD_SAL y PAIS están relacionadas.



## Hipótesis 2
El peso bruto está relacionado de manera directa con el valor total de la mercancía en millones de pesos, entre más pesada sea la mercancía, más va a costar.

In [ ]:
sns.jointplot("FOBPES","PBK",data=df)

In [ ]:
df[["FOBPES","PBK"]].corr(method="pearson")

In [ ]:
df[["FOBPES","PBK"]].corr(method="spearman")

Dado que las variables no están distribuidas normalmente la correlación de Pearson no es la adecuada para determinar qué tan correlacionadas están. Sin embargo está la correlación de Spearman que sirve para determinar y nos indica que hay un 72.8% de correlación entre estas variables

# Impacto en la ley de protección de datos
De acuerdo a los fines del DANE no hay restricciones legales con los datos pues no presentan información sensible y se encuentran a disposición del público para su uso. Sin embargo, hay que ser responsable con los resultados que se obtengan y su publicación.